In [25]:
import pandas as pd
import numpy as np

from scipy import spatial
from sklearn.metrics import accuracy_score
from sklearn.neighbors import BallTree

In [26]:
data = np.genfromtxt('./kmeans_data/data.csv',delimiter=",", dtype=float)
clean_data = data.copy()
labels = np.genfromtxt('./kmeans_data/label.csv',delimiter=",", dtype=int)

In [27]:
def generalized_jaccard(u, v):
    return 1 - np.sum(np.minimum(u,v))/np.sum(np.maximum(u,v))

In [28]:
class Kmeans:
    def __init__(self, K, data):
        indices_for_clusters = np.random.choice(data.shape[0], K) #pick K random initial clusters 
        self.clusters = data[indices_for_clusters] 
        self.cluster_labels = np.arange(0,K,1) 
        self.assignments = np.ones(data.shape[0]) * np.inf  #array to keep track of which cluster each datapoint is assigned to


    def fit(self, data, max_iterations, metric):
        metrics_dict = {"euclidean": spatial.distance.euclidean, "cosine" : spatial.distance.cosine, "jaccard" : generalized_jaccard}
        i=0
        sse = np.inf
        while(1):

            #Assign the datapoints to a cluster
            distances = BallTree(self.clusters, metric=metrics_dict[metric]).query(data)
            nearest_cluster_idx = distances[1]
            self.assignments = np.array([self.cluster_labels[idx] for idx in nearest_cluster_idx]) 
            
            #Recompute the means of each cluster and update the cluster with the mean
            clusters_before_update = self.clusters.copy()
            for idx,centroid in enumerate(self.cluster_labels):
                cluster_members = np.where(self.assignments == centroid)[0]
                cluster_mean = np.mean(data[cluster_members], axis=0) 
                self.clusters[idx] = cluster_mean
            
            #Calculate SSE
            last_iter_sse = sse
            sse = 0
            for idx,centroid in enumerate(self.cluster_labels):
                cluster_members = np.where(self.assignments == centroid)[0]
                sse += np.sum((data[cluster_members] - self.clusters[idx])**2)

            i+=1
            print("iteration: ", i, " SSE :", sse, end='\r')

            #Terminating criteria
            if np.array_equal(self.clusters,clusters_before_update) or i>=max_iterations or sse>last_iter_sse:
                break
        
        predictions = np.ones(data.shape[0]) * np.inf
        for i,_ in enumerate(self.cluster_labels):
            cluster_members = np.where(self.assignments == i)[0]
            counts = np.bincount(labels[cluster_members])
            predictions[cluster_members] = np.argmax(counts)

        return predictions

Possible measure of similarity arguments: 'euclidean', 'cosine', or 'jaccard'

In [29]:
data = clean_data.copy()
kmeans = Kmeans(10,data)
Y_pred = kmeans.fit(data, 100, 'euclidean')

In [30]:
acc = accuracy_score(labels, Y_pred)
acc

0.602